### Building a RAG System with LangChain and FAISS 

pros:
1. Extremely fast similarity search
2. Memory efficient
3. Supports GPU acceleration
4. Can handle millions of vectors

In [1]:
# load libraries
import os
from dotenv import load_dotenv
import numpy as np
from typing import List,Dict,Any
import warnings
warnings.filterwarnings('ignore')

# langChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# langChain specific imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer                #hugging_face Embedding
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader, PyMuPDFLoader
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

load_dotenv()

True

import from pdf files

In [99]:
import re
from pathlib import Path
import unicodedata
class PDFProcessor:
    """complete PDF processing for embedding"""

    def __init__(self, chunk_size=500, chunk_overlap=100):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n","\n"," ",""],
        )

    def preprocess(self, pdf_path: str) -> List[Document]:
        """Load and clean and chunk a single PDF"""
        pdf_path = str(pdf_path)
        file_name = Path(pdf_path).name

        try:
            loader = PyMuPDFLoader(pdf_path)
            pages = loader.load()
        except Exception as e:
            raise RuntimeError(f"Failed to load {pdf_path}: {e}")

        final_chunks = []

        for page_idx, page in enumerate(pages):
            cleaned_text = self._clean_text(page.page_content)

            # skip empty and garbage pages
            if len(cleaned_text) < 50:
                continue
            
            if self._is_garbled_text(cleaned_text):
                print(f"Skipping garbled page {page_idx+1} in {file_name}")
                continue
            
            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[{
                    "file_name": file_name,
                    "source": pdf_path,
                    "page": page_idx + 1,
                    "total_pages": len(pages),
                    "chunk_method": "custom PDFProcessor",
                    "word_count": len(cleaned_text.split()),
                    "char_count": len(cleaned_text),
                }],
            )

            final_chunks.extend(chunks)

        print(f"{file_name}: {len(final_chunks)} chunks created")
        return final_chunks

    def _is_garbled_text(self, text: str) -> bool:
        if not text:
            return True

        # Count control characters
        control_chars = sum(1 for c in text if ord(c) < 32 and c not in "\n\t")

        # Persian letters range
        persian_letters = len(re.findall(r"[آ-ی]", text))

        # Heuristics
        if control_chars > 20:
            return True
        if persian_letters < 10:
            return True
        if persian_letters / max(len(text), 1) < 0.02:
            return True

        return False      
    
    def _clean_text (self,text) -> str:
            # 1. Unicode normalization (SAFE)
        text = unicodedata.normalize("NFKC", text)

        # 2. Remove control characters
        text = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)

        # 3. Normalize whitespace (but keep word boundaries!)
        text = re.sub(r"\s+", " ", text)

        return text.strip()


In [100]:
# we want to extract pdf path from dirloader (dont use loader_cls=TextLoader,)
def get_all_pdf_paths(directory: str) -> list[str]:
    loader = DirectoryLoader( # didnt work bcz try to use default UnstructuredPDFLoader
                              # which depend on other libraries to be installed (slow and bad results in persian pdfs)
        directory,
        glob="**/*.pdf",
        show_progress=True
    )
    docs = loader.load()

    # extract unique file paths
    paths = list({doc.metadata["source"] for doc in docs})
    return paths

In [101]:
# new rubust and easy way to get paths
def new_get_all_pdf_paths(directory: str) -> list[str]:
    
    return [str(p) for p in Path(directory).rglob("*.pdf")]

In [102]:
def load_and_process_pdfs (data_dir: str) -> list[Document]:
    processor = PDFProcessor(
        chunk_size=500,
        chunk_overlap=100
    )

    pdf_paths = new_get_all_pdf_paths(data_dir)
    print(f"Found {len(pdf_paths)} PDFs")

    all_chunks = []

    for pdf_path in pdf_paths:
        chunks = processor.preprocess(pdf_path)
        all_chunks.extend(chunks)

    print(f"total chunks created: {len(all_chunks)} from {len(pdf_paths)} pdfs")
    return all_chunks

In [103]:
documents = load_and_process_pdfs("G:/extracted/ML/RAG/data_management/pdf_files")

Found 6 PDFs
آموزش بازی بیلیارد.pdf: 15 chunks created
MuPDF error: format error: No default Layer config

آموزش_تعمیر_اتومبیلهای_سواری.pdf: 55 chunks created
ابلاغیه.pdf: 0 chunks created
Skipping garbled page 1 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 2 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 3 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 4 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 5 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 6 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 7 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 8 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 9 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 10 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 11 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 12 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 13 in مدار_گاز_سوز_اتوبوس_های_شهری.pdf
Skipping garbled page 14 in

In [104]:
print(documents[55].page_content)

آن است، تـاب برداشـتن اسـت و بـراي مشخص كردن مي بايست از يك خط كش فلزي استفاده كنيم . - عيب ديگر : پيچيدگي در سر سيلندر است كه تا حدودي قابل تعميرا ست در صورتيكه بيشـتر از اندازه مجاز باشد قابل تعمير نيست مثلاً از حدود 20 /0 ميليمتر بيشتر باشد قابـل تعميـر نخواهـد .بود عيب ديگر : مربوط به فنر سوپاپ در سرسيلندر است، مي بايست فنرهاي سوپاپ هوا و دود را در كنار هم قرار دهيم و مي بايست همگي يك انازه باشند . در صو رت كوتـاه بـودن آن فنـري كـه كوتاه بود قابل تعمير نيست و باعث سوختن سوپاپ مي شود . واشر


### Embedding using HooshvareLab/distilbert-fa-zwnj-base

In [73]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(
    model_name="HooshvareLab/distilbert-fa-zwnj-base",
    model_kwargs={"device": "cpu"},   # or cuda for gpu
    encode_kwargs={
        "normalize_embeddings": True,  # so important for cosine similarity
        "batch_size": 32
    }
)
embeddings = embedding_function.embed_documents([documents[20].page_content])[0]
print("First 10 values:", embeddings[:5])

No sentence-transformers model found with name HooshvareLab/distilbert-fa-zwnj-base. Creating a new one with mean pooling.


First 10 values: [0.02403348870575428, -0.01216572429984808, 0.023973509669303894, -0.006327626761049032, 0.011993647553026676]


In [105]:
# multilingual hugging face
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_function2 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [106]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_function2
)

print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 70 vectors


In [87]:
vectorstore.save_local("faiss_db")

In [88]:
vectorstore.load_local(
    "faiss_db",
    embeddings=embedding_function,
    allow_dangerous_deserialization=True
    )

In [ ]:
similar = vectorstore.similarity_search(
    "فنرهاي سوپاپ هوا و دود را در كنار هم قرار دهيم",
    k=3,
    )
similar

[Document(id='e08f8969-349d-4d3e-b42e-a0566f99f5ac', metadata={'file_name': 'آموزش_تعمیر_اتومبیلهای_سواری.pdf', 'source': 'G:\\extracted\\ML\\RAG\\data_management\\pdf_files\\آموزش_تعمیر_اتومبیلهای_سواری.pdf', 'page': 11, 'total_pages': 22, 'chunk_method': 'custom PDFProcessor', 'word_count': 368, 'char_count': 1672}, page_content='بـه ايـن حركـت در بالاي خود يك حجمي را ايجاد مي كند كه اين حجم ايجاد شـده و خـلاء حاصـله و همينطـور زمانبندي كه براي سوپاپها در نظر گرفته شده، سوپاپ هوا باز مي شود و بـا بـاز شـدن سـوپاپ هوا، خلا حاصله در نتيجه پائين رفتن پيستون ، توسط مخلوط سوخت و هوايي كه كاربراتور انجم داده، به فضاي بالاي پيستون راه پيدا مي كند و مرحله مكش انجام مي گيرد . زمان دوم : با ادامه حركت ميل لنگ و گردش به سمت بـالا، پيسـتون هـم بـه سـمت بـالا حركت داده مي شود و هم زمان با اين عمل سوپاپ ورودي كه در مرحله مكش بـاز'),
 Document(id='336c4479-602a-48b8-9069-4ae261b4385b', metadata={'file_name': 'آموزش_تعمیر_اتومبیلهای_سواری.pdf', 'source': 'G:\\extracted\\ML\\RAG\\data_management\\pdf

In [120]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""بر اساس متن اطلاعات موجود به سوال جواب بده اگر جواب را پیدا نکردی بگو نمیدانم
اطلاعات: {context}

سوال: {question}

جواب:""")

In [109]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020534F732F0>, search_kwargs={'k': 3})

In [110]:
# using modern lcel
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [111]:
from langchain_groq import ChatGroq
from langchain.chat_models.base import init_chat_model
load_dotenv()
os.environ["GROQ_API"]=os.getenv("GROQ_API")
groq_llm = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API"))

In [113]:
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }
    | simple_prompt
    | groq_llm
    |StrOutputParser()

)

In [119]:
# Conversational RAg Chain
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "بر اساس اطلاعات موجود در متن داده شده به سوال جواب بده"),
    ("placeholder", "{chat_history}"),
    ("human", "اطلاعات: {context}\n\n سوال: {input}"),
])

In [115]:
def create_conversational_rag():
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | groq_llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [116]:
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | groq_llm
)

In [117]:
def test_rag_chains(question: str):
    """Test all RAG chain """
    print(f"Question: {question}")
    print("=" * 80)
    
    #  Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [122]:
test_rag_chains("تعداد توپ های بیلیارد و توپ های خط دار")

Question: تعداد توپ های بیلیارد و توپ های خط دار

1. Simple RAG Chain:
Answer: بر اساس اطلاعات موجود در سند 1، تعداد توپ های بیلیارد از 1 تا 51 است. همچنین، بر اساس سند 2، توپ های خط دار از شماره 9 تا 51 تشکیل می شوند.

2. Streaming RAG:
Answer: بر اساس اطلاعات موجود در Documents 1 و 2، تعداد توپ های بیلیارد از شماره 1 تا 151 است. از این تعداد، توپ های ساده از شماره 1 تا 7 و توپ های خط دار از شماره 9 تا 51 را تشکیل می دهند. همچنین، توپ 8 خاص است و جزو هیچ گروهی محسوب نمی شود.

بنابراین، تعداد توپ های بیلیارد 151 نفر است و تعداد توپ های خط دار 43 نفر است (از شماره 9 تا 51).


In [123]:
documents[1].page_content

'خوب به رنگ ها نگاه يد آن , نخستين چيز آه بايد درباره ي بيلي يارد بدانيد اين است آه اين ب ازي برخلاف باري ها ي توپي ديگر داراي توپ هاي زيادي است . برخي از اين توپ ها ساده يعني داراي رنگي يك پارچه هستند و تعدادي از آنها نيز خط دارند يعني توپي آه نوك و ته آن ها سفيد و يك خط رنگي آه گرداگرد و ميانه ي آن قرار گرفته . هر يك از توپ ها دار اي يك شماره هستند . از شماره ي تا ١ ٥١ , آه از شماره ي ١ تا ٧ تو پ هاي ساده واز شماره ي ٩ تا ٥١ توپ هاي خط دارد را تشكيل مي دهند . توپ شماره ي ٨ ٨ballآدام است ؟ .'